In [4]:
!python --version

Python 3.6.12


In [9]:
!python -m spacy download en_core_web_lg

     |████████████████████████████▍   | 732.9 MB 1.2 MB/s eta 0:01:20

     |████████████████████████████████| 826.9 MB 637 kB/s eta 0:00:01     |████████████████████████████▊   | 741.8 MB 142 kB/s eta 0:09:57     |█████████████████████████████   | 751.4 MB 821 kB/s eta 0:01:32     |█████████████████████████████▏  | 752.6 MB 266 kB/s eta 0:04:39     |█████████████████████████████▍  | 758.9 MB 1.5 MB/s eta 0:00:45     |█████████████████████████████▍  | 759.2 MB 1.5 MB/s eta 0:00:45     |█████████████████████████████▍  | 760.8 MB 1.1 MB/s eta 0:00:59     |██████████████████████████████▏ | 778.2 MB 704 kB/s eta 0:01:10     |██████████████████████████████▏ | 778.7 MB 768 kB/s eta 0:01:03     |███████████████████████████████▏| 804.1 MB 111 kB/s eta 0:03:26     |███████████████████████████████▋| 817.2 MB 717 kB/s eta 0:00:14     |███████████████████████████████▉| 822.4 MB 1.0 MB/s eta 0:00:05
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


need to install py2neo, requests, spacy, en_core_web_sm. Then also get an eventregistry.org api key

Remove key before git commit. Put key back before running.

install pandas for dataframes

Following along with this: https://medium.com/analytics-vidhya/build-a-knowledge-graph-using-neo4j-eb7490799f4b

In [10]:
import pickle
import requests
import json
from py2neo import database, Graph, Node, Relationship
import re
import en_core_web_sm
import os
import random
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

nlp = en_core_web_sm.load()

article_uri = dict()
URL = "http://eventregistry.org/api/v1/article/getArticles"
KEYWORDS = [["antiquities trade", "illicit antiquities", "illegal antiquties", "art trade", "art market"]]


def clear_graph(graph):
    print("[+] Clearing previous graph")
    # graph.cypher.execute("MATCH (A) -[R] -> () DELETE A, R")
    # graph.cypher.execute("MATCH (A) DELETE A")
    graph.delete_all()
    file_name = "Entities/entities.json"
    if os.path.isfile(file_name):
        os.remove(file_name)


def get_all_entites():
    entities = dict()

    entities["news_sources"] = dict()
    entities["authors"] = dict()
    entities["languages"] = dict()
    entities["organisations"] = dict()
    entities["locations"] = dict()
    entities["people"] = dict()
    entities["articles"] = dict()

    file_name = "Entities/entities.json"
    if os.path.isfile(file_name):
        with open(file_name) as f:
            entities = json.load(f)

    return entities


def get_articles_request(keywords, num_articles=100):
    print("[+] Getting articles for keywords:", keywords)
    articles = []

    
    # for keywords in KEYWORDS:
    payload = {"action": "getArticles", "keyword": keywords, "articlesPage": 1, "articlesSortBy": "date",
               "articlesSortByAsc": False, "articlesArticleBodyLen": -1, "resultType": "articles",
               "dataType": ["news", "pr"], "apiKey": "9ab7140b-1113-4329-a560-97dfb9970cde",
               "forceMaxDataTimeWindow": 31,
               "articlesCount": num_articles}

    headers = {
        'Content-Type': 'application/json'
    }

    response = requests.request("POST", URL, headers=headers, data=json.dumps(payload))
    try:
        articles = json.loads(response.text.encode('utf8'))
    except:
        print("Exception occured for keyword:", keywords)
        return list()
    # print(articles.keys())
    print(type(articles))
    print("Articles request len", len(articles["articles"]["results"]))
    return articles["articles"]["results"]


def get_articles(num_articles=100):
    curr_date = datetime.today().strftime('%d-%m-%Y')
    #curr_date = "29 -05-2020"
    file_name = "Daily_Articles/" + curr_date + ".pickle"

    if not os.path.isfile(file_name):
        print("\tCould not find articles at: {file_name}, requesting".format(file_name=file_name))
        with open(file_name, "wb") as handle:
            articles = list()
            for keywords in KEYWORDS[1]:
                articles += get_articles_request(keywords, num_articles)
            articles_dict = dict()
            articles_dict["articles"] = {"results":articles}
            pickle.dump(articles_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
            return articles_dict #i changed this b/c of twitter guy
    else:
        print("\tFound articles at: {file_name}".format(file_name=file_name))
        with open(file_name, 'rb') as handle:
            return pickle.load(handle)


def update_entities(entities):
    print("[+] Updating entities")

    # with open('Entities/entities.pkl', 'wb') as output:
    #     pickle.dump(entities, output, pickle.HIGHEST_PROTOCOL)

    file_name = "Entities/entities.json"
    with open(file_name, 'w') as fp:
        json.dump(entities, fp)


def get_random_number():
    return random.randint(100000000, 1000000000)


def check_wiki_page(query):
    try:
        result = requests.get('https://en.wikipedia.org/wiki/{0}'.format(query), verify=False)
    except Exception:
        pass

    if result.status_code == 200:  # the article exists
            return True
    return False


def create_graph(articles, entities, graph):
    count_ = 0
    articles_uri = entities["articles"]
    news_sources = entities["news_sources"]
    languages = entities["languages"]
    authors = entities["authors"]
    people = entities["people"]
    locations = entities["locations"]
    organisations = entities["organisations"]

    new_articles = []

    print("[+] Creating nodes and relationships for languages, articles, authors and content-sources")

    for article_ in articles:#["articles"]["results"]:
        count_ += 1
        if article_["uri"] not in articles_uri:
            new_articles.append(article_)
            print("\tEntering data for article:", article_["uri"], "- title:", article_["title"])

            # Creating Nodes
            article_pk = get_random_number()
            article_node = Node("Article", pk=article_pk, name=article_['title'], sentiment=article_["sentiment"],
                                text=article_["body"],
                                dtype=article_["dataType"], published_on=article_["dateTimePub"], uri=article_["uri"],
                                url=article_["url"])

            articles_uri[article_["uri"]] = (article_pk, article_node)

            # Entering article
            if article_["source"]["title"] in news_sources:
                source_node_pk = news_sources[article_["source"]["title"]][0]
                query = "Match(m:`Content-Source`) where m.pk={0} Return m".format(source_node_pk)
                source_node = graph.evaluate(query)
            else:
                source_node_pk = get_random_number()
                source_node = Node("Content-Source", pk=source_node_pk, name=article_["source"]["title"],
                                   uri=article_["source"]["uri"])

            rel_article_source = Relationship(article_node, "publishedBy", source_node)
            graph.create(rel_article_source)
            news_sources[article_["source"]["title"]] = (source_node_pk, source_node)

            #Entering language
            if article_["lang"] in languages:
                lang_pk = languages[article_["lang"]][0]
                query = "Match(m:`Language`) where m.pk={0} Return m".format(lang_pk)
                lang_node = graph.evaluate(query)
            else:
                lang_pk = get_random_number()
                lang_node = Node("Language", pk=lang_pk, name=article_["lang"])
            
            rel_article_lang = Relationship(article_node, "language", lang_node)
            graph.create(rel_article_lang)
            languages[article_["lang"]] = (lang_pk, lang_node)

            # Entering authors
            for author in article_["authors"]:
                if author["name"] in authors:
                    author_pk = authors[author["name"]][0]
                    query = "Match(m:`Author`) where m.pk={0} Return m".format(author_pk)
                    author_node = graph.evaluate(query)
                else:
                    author_pk = get_random_number()
                    author_node = Node("Author", pk=author_pk, name=author["name"], uri=article_["uri"])

                rel_article_author = Relationship(article_node, "writtenBy", author_node)
                rel_author_article = Relationship(author_node, "hasWritten", article_node)

                graph.create(rel_article_author)
                graph.create(rel_author_article)

                authors[author["name"]] = (author_pk, author_node)

    print("[+] Done loading data to neo4j")

    entities["news_sources"] = news_sources
    entities["authors"] = authors
    entities["languages"] = languages
    entities["articles"] = articles_uri
    entities["people"] = people
    entities["locations"] = locations
    entities["organisations"] = organisations

    # update_entities(entities)

    return entities, new_articles


def create_graph_with_parsed_entites(articles, entities, graph, valid_entities, invalid_entities):
    people = entities["people"]
    locations = entities["locations"]
    organisations = entities["organisations"]
    articles_uri = entities["articles"]
    news_sources = entities["news_sources"]
    languages = entities["languages"]
    authors = entities["authors"]

    # entity_type_map = {"ORG": "organisations", "PERSON": "people", "GPE": "location"}

    
    # documentation for entities recognized by spacy: https://spacy.io/api/annotation#named-entities
    # PRODUCT and WORK_OF_ART see if you can add this
    print("[+] Creating nodes and relationships for people, locations and organisations")
    print(len(articles))

    for i, article_ in enumerate(articles):
        print(i)
        start_time = time.time()

        print("\tEntering data for article:", article_["uri"], "- title:", article_["title"])

        article_pk = articles_uri[article_["uri"]][0]

        text_nlp = nlp(article_["body"])

        for ent in text_nlp.ents:

            if "%" in ent.text:
                continue

            ent_text = ent.text.lower()
            ent_text = ent_text.replace("-", "").replace("'s", "")

            if not check_wiki_page(ent_text):
                continue

            if ent.label_ is "ORG":
                if ent_text in organisations:
                    entity_pk = organisations[ent_text][0]
                    query = "Match(m:`ORGANISATION`) where m.pk={0} Return m".format(entity_pk)
                    entity_node = graph.evaluate(query)
                else:
                    entity_pk = get_random_number()
                    entity_node = Node("ORGANISATION", pk=entity_pk, name=ent_text)

                query = "Match(m:`Article`) where m.pk={0} Return m".format(article_pk)
                article_node = graph.evaluate(query)

                rel_article_org = Relationship(article_node, "referencesOrg", entity_node)
                graph.create(rel_article_org)

                organisations[ent_text] = (entity_pk, entity_node)

            elif ent.label_ is "PERSON":
                if ent_text in people:
                    entity_pk = people[ent_text][0]
                    query = "Match(m:`Person`) where m.pk={0} Return m".format(entity_pk)
                    entity_node = graph.evaluate(query)
                else:
                    entity_pk = get_random_number()
                    entity_node = Node("Person", pk=entity_pk, name=ent_text)

                query = "Match(m:`Article`) where m.pk={0} Return m".format(article_pk)
                article_node = graph.evaluate(query)

                rel_article_person = Relationship(article_node, "referencesPerson", entity_node)
                graph.create(rel_article_person)

                people[ent_text] = (entity_pk, entity_node)

            elif ent.label_ is "GPE":
                if ent_text in locations:
                    entity_pk = locations[ent_text][0]
                    query = "Match(m:`Location`) where m.pk={0} Return m".format(entity_pk)
                    entity_node = graph.evaluate(query)
                else:
                    entity_pk = get_random_number()
                    entity_node = Node("Location", pk=entity_pk, name=ent_text)

                query = "Match(m:`Article`) where m.pk={0} Return m".format(article_pk)
                article_node = graph.evaluate(query)

                rel_article_loc = Relationship(article_node, "referencesLoc", entity_node)
                graph.create(rel_article_loc)

                locations[ent_text] = (entity_pk, entity_node)
    
            else:
                continue
            # valid_entities.add((article_["uri"], ent.text, ent.label_))

        print("--- %s seconds ---" % (time.time() - start_time))

    entities["people"] = people
    entities["locations"] = locations
    entities["organisations"] = organisations

    return entities





FileNotFoundError: [Errno 2] No such file or directory: '/opt/anaconda3/envs/yates/lib/python3.6/site-packages/en_core_web_sm/en_core_web_sm-2.3.1/vocab/lexemes.bin'

In [6]:
curr_date = datetime.today().strftime('%d-%m-%Y')
filename = "Daily_Articles/" + curr_date + ".pickle"
outfile = open(filename,'wb')
stuff = get_articles_request('antiquities trade')
more_stuff = get_articles_request('illicit antiquities')
even_more_stuff = get_articles_request('illegal antiquities')
still_more_stuff = get_articles_request('art market')
encore_more_stuff = get_articles_request('art auction')
sothebys_more_stuff = get_articles_request('sothebys')
christies_more_stuff = get_articles_request('christies')
pickle.dump(stuff,outfile)
pickle.dump(even_more_stuff,outfile)
pickle.dump(even_more_stuff,outfile)
pickle.dump(still_more_stuff,outfile)
pickle.dump(encore_more_stuff,outfile)
pickle.dump(sothebys_more_stuff,outfile)
pickle.dump(christies_more_stuff,outfile)
outfile.close()

NameError: name 'get_articles_request' is not defined

In [4]:
if __name__ == "__main__":
    graph = Graph(password="cows")
    valid_entities, invalid_entities = dict(), dict()
    #clear_graph(graph)
    articles = get_articles()
    entities = get_all_entites()
    time.sleep(2)
    entities, new_articles = create_graph(articles, entities, graph)
    print(len(new_articles))
    entities = create_graph_with_parsed_entites(new_articles, entities, graph, valid_entities, invalid_entities)
    update_entities(entities)

	Found articles at: Daily_Articles/10-02-2021.pickle
[+] Creating nodes and relationships for languages, articles, authors and content-sources
	Entering data for article: 6425348305 - title: Forget bandages: This first-ever Egyptian mud-wrapped mummy dates
	Entering data for article: 6425232200 - title: Khaled al-Asaad: Possible remains of Syrian archaeologist found at Palmyra * The Syrian Observatory For Human Rights
	Entering data for article: 6424852820 - title: Khaled al-Asaad: Possible remains of Syrian archaeologist found at Palmyra
	Entering data for article: 6421553646 - title: Museum of the Bible Returns Looted Artifacts to Egypt and Iraq
	Entering data for article: 6420093308 - title: Who Changed The Bodies and What is The Real Name of The Egyptian Mummy? | Al Bawaba
	Entering data for article: 6419464515 - title: Rare Ancient Egyptian 'mud mummy' may have been in wrong coffin all this time
	Entering data for article: 6419453281 - title: Rare Ancient Egyptian 'mud mummy' may 

Goddamn, it works! go over to the neo4j browser and run ```match (n) return n```

Now, I wonder if I can get the other thing (https://colab.research.google.com/drive/1-coQe5FXN30BAXTSE6Rt3fKRAgdEuoUl?usp=sharing) to process the actual text? this seems to only parse the existing metadata. Note that there are sentiment scores, a bunch of different metadata